In [1]:
import requests
import json
import pandas as pd
import librosa
import numpy as np
import lyricsgenius
import langdetect
import re
import tempfile
import sys,os,os.path
import requests
from bs4 import BeautifulSoup
from collections import deque
import numpy as np
import pandas as pd
import datetime as dt
import ast
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

LASTFM_API_KEY = os.environ['lastfm_api_key']
LASTFM_SECRET = os.environ['lastfm_secret']
GENIUS_CLIENT_ID = os.environ['genius_client_id']
GENIUS_CLIENT_SECRET = os.environ['genius_client_secret']
GENIUS_ACCESS_TOKEN = os.environ['genius_access_token']

### Last.FM API for user data

In [57]:
base_url = 'http://ws.audioscrobbler.com/2.0/'

def lastfm_get(payload):
    headers = {'user-agent': 'DataCollectorBot'}
    payload['api_key'] = LASTFM_API_KEY
    payload['format'] = 'json'
    response = requests.get(base_url, headers=headers, params=payload)
    return response.json()

# Function for each endpoint
def get_user_info(user):
    payload = {'method': 'user.getinfo', 'user': user}
    return lastfm_get(payload)

def get_loved_tracks(user):
    payload = {'method': 'user.getlovedtracks', 'user': user}
    return lastfm_get(payload)

def get_personal_tags(user):
    """user (Required) : The user who performed the taggings.
    tag (Required) : The tag you're interested in.
    taggingtype[artist|album|track] (Required) : The type of items which have been tagged
    ...
    """
    payload = {'method': 'user.getPersonalTags', 'user': user}
    return lastfm_get(payload)

def get_recent_tracks(user):
    payload = {'method': 'user.getrecenttracks', 'user': user}
    return lastfm_get(payload)

def get_top_albums(user):
    payload = {'method': 'user.gettopalbums', 'user': user}
    return lastfm_get(payload)

def get_top_artists(user):
    payload = {'method': 'user.gettopartists', 'user': user}
    return lastfm_get(payload)

def get_top_tags(user):
    payload = {'method': 'user.gettoptags', 'user': user}
    return lastfm_get(payload)

def get_top_tracks(user):
    payload = {'method': 'user.gettoptracks', 'user': user}
    return lastfm_get(payload)

def get_weekly_album_chart(user):
    payload = {'method': 'user.getweeklyalbumchart', 'user': user}
    return lastfm_get(payload)

def get_weekly_artist_chart(user):
    payload = {'method': 'user.getweeklyartistchart', 'user': user}
    return lastfm_get(payload)

def get_weekly_chart_list(user):
    payload = {'method': 'user.getweeklychartlist', 'user': user}
    return lastfm_get(payload)

def get_weekly_track_chart(user):
    payload = {'method': 'user.getweeklytrackchart', 'user': user}
    return lastfm_get(payload)

def get_song_details(artist_name, track_name):
    payload = {
        'method': 'track.getInfo',
        'artist': artist_name,
        'track': track_name,
    }
    json_response = lastfm_get(payload)
    return json_response.get('track', {})

# username = 'joehoots'
# user_data = {
#     'info': get_user_info(username),
#     'loved_tracks': get_loved_tracks(username),
#     # 'personal_tags': get_personal_tags(username),
#     'recent_tracks': get_recent_tracks(username),
#     'top_albums': get_top_albums(username),
#     'top_artists': get_top_artists(username),
#     'top_tags': get_top_tags(username),
#     'top_tracks': get_top_tracks(username),
#     # 'weekly_album_chart': get_weekly_album_chart(username),
#     # 'weekly_artist_chart': get_weekly_artist_chart(username),
#     # 'weekly_chart_list': get_weekly_chart_list(username),
#     # 'weekly_track_chart': get_weekly_track_chart(username)
# }

artist_name = "Harlow Road"
track_name = "Clock Hands"
get_song_details(artist_name, track_name)


{'name': 'Clock Hands',
 'url': 'https://www.last.fm/music/Harlow+Road/_/Clock+Hands',
 'duration': '167000',
 'streamable': {'#text': '0', 'fulltrack': '0'},
 'listeners': '4200',
 'playcount': '38345',
 'artist': {'name': 'Harlow Road',
  'url': 'https://www.last.fm/music/Harlow+Road'},
 'album': {'artist': 'Harlow Road',
  'title': 'Clock Hands',
  'url': 'https://www.last.fm/music/Harlow+Road/Clock+Hands',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/958b54524dc4bb5a128df236de637c2b.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/958b54524dc4bb5a128df236de637c2b.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/958b54524dc4bb5a128df236de637c2b.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/958b54524dc4bb5a128df236de637c2b.png',
    'size': 'extralarge'}]},
 'toptags': {'tag': [{'name': 'trap', 'url': 'https://www.last.fm/tag/trap'},
   {'name': 'pop r

In [3]:
user_data

{'info': {'user': {'name': 'joehoots',
   'age': '0',
   'subscriber': '0',
   'realname': 'Hoots',
   'bootstrap': '0',
   'playcount': '130988',
   'artist_count': '1184',
   'playlists': '0',
   'track_count': '16889',
   'album_count': '2628',
   'image': [{'size': 'small',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/34s/41c9518e2213898c127494f6746ec3b1.png'},
    {'size': 'medium',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/64s/41c9518e2213898c127494f6746ec3b1.png'},
    {'size': 'large',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/174s/41c9518e2213898c127494f6746ec3b1.png'},
    {'size': 'extralarge',
     '#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/41c9518e2213898c127494f6746ec3b1.png'}],
   'registered': {'unixtime': '1484237303', '#text': 1484237303},
   'country': 'United Kingdom',
   'gender': 'n',
   'url': 'https://www.last.fm/user/joehoots',
   'type': 'user'}},
 'loved_tracks': {'lovedtracks': {'track': [{'artist': {'url': 'https:/

In [20]:
# TESTING
infokeys = []
for k in user_data.keys():
    infokeys.extend(user_data[k].keys())
infokeys

# {'message': 'User not found', 'error': 6},
# 'JekabsBalodis', 'Bans77', 'BloodOnLeaves', 'chippy_boi', 'joehoots', ....

['user',
 'lovedtracks',
 'recenttracks',
 'topalbums',
 'topartists',
 'toptags',
 'toptracks']

In [ ]:
recent_tracks = user_data['recent_tracks']

# Print each track with its listen timestamp
for track in recent_tracks['recenttracks']['track']:
    track_name = track['name']
    artist_name = track['artist']['#text']
    # Check if the 'date' key exists
    if 'date' in track:
        listen_time = track['date']['#text']  # Human-readable timestamp
        listen_timestamp = track['date']['uts']  # Unix timestamp
        print(f"{artist_name} - {track_name} listened to at {listen_time} (timestamp: {listen_timestamp})")
    else:
        # If 'date' key does not exist, it might be currently playing
        print(f"{artist_name} - {track_name} is currently playing or missing timestamp data.")

### Constructing the main user-song dataset

In [3]:
min_users = 10000
min_songs = 100000
users_songs = {}  # This will store user: {"top_50": [], "loved": []}
all_songs = set()

def get_lastfm_data(start_username, api_key):
    discovered = set([start_username])
    queue = deque([start_username])

    while queue and (len(users_songs) < min_users or len(all_songs) < min_songs):
        current_user = queue.popleft()
        
        friends_url = f"http://ws.audioscrobbler.com/2.0/?method=user.getfriends&user={current_user}&api_key={api_key}&format=json"
        try:
            friends_response = requests.get(friends_url)
            friends_data = friends_response.json()
            
            if 'error' in friends_data:
                print(f"Error fetching friends for user {current_user}: {friends_data['message']}")
                continue
            
            for user in friends_data.get('friends', {}).get('user', []):
                friend_name = user['name']
                if friend_name not in discovered:
                    discovered.add(friend_name)
                    queue.append(friend_name)

                    loved_tracks_url = f"http://ws.audioscrobbler.com/2.0/?method=user.getlovedtracks&user={friend_name}&api_key={api_key}&format=json"
                    top_tracks_url = f"http://ws.audioscrobbler.com/2.0/?method=user.gettoptracks&user={friend_name}&api_key={api_key}&format=json&limit=50"
                    # recent_tracks_url = f"http://ws.audioscrobbler.com/2.0/?method=user.getrecenttracks&user={friend_name}&api_key={api_key}&format=json&limit=50"
                    
                    loved_tracks_response = requests.get(loved_tracks_url)
                    top_tracks_response = requests.get(top_tracks_url)
                    # recent_tracks_response = requests.get(recent_tracks_url)

                    user_info = get_user_info(friend_name)
                    loved_tracks_data = loved_tracks_response.json()
                    top_tracks_data = top_tracks_response.json()
                    # recent_tracks_data = recent_tracks_response.json()
                    
                    # user exists
                    if (len(user_info) == 1) and ('error' not in loved_tracks_data) and ('error' not in top_tracks_data):
                        # and ('error' not in recent_tracks_data)

                        # 'name', 'artist' for loved_tracks, may be empty
                        loved_tracks = [(track['name'], track['artist']['name']) for track in loved_tracks_data.get('lovedtracks', {}).get('track', [])]

                        # ['name', 'artist', 'rank', 'playcount'] for top_track
                        top_tracks = [(track['name'], track['artist']['name'], track['@attr']['rank'], track['playcount']) for track in top_tracks_data.get('toptracks', {}).get('track', [])]
                        update_top = [(item[0], item[1]) for item in top_tracks]
                       
                        #    """taking too long, TODO it seperately"""
                        # ['name', 'artist', 'date']  for recent_tracks, date could be empty
                        # recent_tracks = [(track['name'], track['artist']['#text'], track['date']['#text']) for track in recent_tracks_data.get('recenttracks', {}).get('track', [])]
                        # update_recent = [(item[0], item[1]) for item in recent_tracks]

                        users_songs[friend_name] = {"user_info": user_info.get('user', []), "top_50": top_tracks, "loved": loved_tracks, 
                                                    # "recent50": recent_tracks
                                                    }

                        all_songs.update(loved_tracks, update_top, 
                                        #  update_recent
                                         )

            print(f"Collected {len(users_songs)} users and {len(all_songs)} unique songs so far...")

        except Exception as e:
            print(f"An error occurred while processing user {current_user}: {e}")
            continue
        
    df = pd.DataFrame([(user, data['user_info'], data['top_50'], data['loved'], 
                        # data['recent50']
                        ) for user, data in users_songs.items()], columns=['Username', 'User Info', 'Top 50 Songs', 'Liked Songs', 
                                                                        #    'Recent Songs'
                                                                           ])
    
    # print(len(users_songs))
        
    return users_songs, df

In [4]:
%%time
# 313m 20.5s

user_songs, df = get_lastfm_data("chippy_boi", LASTFM_API_KEY)

# start_username = ['JekabsBalodis', 'Bans77', 'BloodOnLeaves', 'chippy_boi', 'joehoots', "bouquet-of-sun"]


# saving the raw dataset
df.to_csv('lastfm_user_raw.csv', index=False)
# print("Data saved to lastfm_user_raw.csv")


Collected 33 users and 2733 unique songs so far...
Collected 40 users and 3141 unique songs so far...
Collected 74 users and 5402 unique songs so far...
Error fetching friends for user riddy_pr: no such page
Error fetching friends for user TheGirlNamedSig: no such page
Collected 124 users and 8816 unique songs so far...
Collected 125 users and 8864 unique songs so far...
Collected 136 users and 9466 unique songs so far...
Error fetching friends for user Brotendo: no such page
Collected 184 users and 12504 unique songs so far...
Collected 233 users and 15399 unique songs so far...
Collected 281 users and 18535 unique songs so far...
Collected 331 users and 21259 unique songs so far...
Collected 381 users and 24310 unique songs so far...
Collected 416 users and 26250 unique songs so far...
Collected 459 users and 28483 unique songs so far...
Collected 507 users and 31591 unique songs so far...
Collected 518 users and 32009 unique songs so far...
Collected 568 users and 34308 unique songs

### cleaning

In [9]:
# cleaning user-song dataset

df = pd.read_csv('lastfm_user_raw.csv', index_col = None)
df.columns = [
            'Username',
            'User Info',
            'Top_50', # (trackname, artistname, rank, playcount)',
            'Loved' # (trackname, artistname)'
            ]

def safe_eval(x):
    try:
        return ast.literal_eval(x)
    except:
        return x
    
df['User Info'] = df['User Info'].apply(safe_eval)
df_info = pd.json_normalize(df['User Info'])
df = pd.concat([df.drop('User Info', axis = 1), df_info], axis = 1).drop(['name', 'image', 'url'], axis = 1)

# convert unix to datetime
df['registered.#text'] = df['registered.#text'].apply(lambda stamp: dt.datetime.fromtimestamp(stamp))
df['registered_year'] = df['registered.#text'].dt.year
df['registered_month'] = df['registered.#text'].dt.month
df['registered_day'] = df['registered.#text'].dt.day

# drop users with no song information - (may be inactive users or cold start users)
empty_songs = df[(df['Top_50'] == "[]") & (df['Loved'] == "[]")]
df = df.drop(empty_songs.index)

print("number of unique values: \n", df.nunique())
# drop columns which are not useful
df = df.drop(['age', 'subscriber', 'realname', 'bootstrap', 'playlists', 'gender', 'registered.#text', 'registered.unixtime'], axis = 1).reset_index(drop=True)

# df[(df['Top_50'] == "[]")]

# saving the clean dataset
df.to_csv('lastfm_user_clean.csv', index = False)
# df.head()

number of unique values: 
 Username               10005
Top_50                 10000
Loved                   7745
age                        1
subscriber                 2
realname                6370
bootstrap                  1
playcount               9756
artist_count            5501
playlists                  1
track_count             8797
album_count             7063
country                  158
gender                     1
type                       4
registered.unixtime    10004
registered.#text       10004
registered_year           23
registered_month          12
registered_day            31
dtype: int64


### constructing unique songs dataset

In [113]:
# getting unique tracks
df['Top_50'] = df['Top_50'].apply(safe_eval)
df['Loved'] = df['Loved'].apply(safe_eval)
# df[df['Loved'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
# 2256 users without loved songs
# df[df['Top_50'].apply(lambda x: isinstance(x, list) and len(x) == 0)]
# zero = df[df['Top_50'].apply(lambda x: len(x) == 0)]
# zero['Top_50'].apply(lambda row: [(tup[0], tup[1]) for tup in row])
# 6 users without top 50 songs

unique_songs_top = df['Top_50'].apply(lambda row: [(tup[0], tup[1]) for tup in row])
unique_songs = [item for sublist in unique_songs_top for item in sublist] + [item for sublist in df['Loved'] for item in sublist]
print(len(unique_songs))
unique_songs = set(unique_songs)
print(len(unique_songs))

unique_songs = pd.DataFrame(unique_songs, columns=['track_name', 'artist_name'])
unique_songs = unique_songs.dropna().reset_index(drop=True)
print(unique_songs.shape)
unique_songs.to_csv('unique_songs.csv',  encoding='utf-8', index=False)

809818
335264
(335264, 2)


In [121]:
# adding song features
unique_songs = pd.read_csv('unique_songs.csv',  encoding='utf-8', dtype=str, na_values = [], na_filter = False)
unique_songs.isna().sum()

track_name     0
artist_name    0
dtype: int64

In [ ]:
# keeping only english songs


In [ ]:
# final user-englishsong mapping in long format

## another version of user-song using different starting user (can ignore for now)

In [8]:
# # can ignore first
# collecting another batch using starting from bouquet-of-sun

%%time
min_users = 10000
min_songs = 100000
users_songs = {}  # This will store user: {"top_50": [], "loved": []}
all_songs = set()
user_songs2, df2 = get_lastfm_data("bouquet-of-sun", LASTFM_API_KEY)
df2.to_csv('lastfm_user_raw2.csv', index = False)

Collected 32 users and 2475 unique songs so far...
Collected 82 users and 6148 unique songs so far...
Collected 130 users and 9646 unique songs so far...
An error occurred while processing user aria-amethyst: HTTPConnectionPool(host='ws.audioscrobbler.com', port=80): Max retries exceeded with url: /2.0/?method=user.getinfo&user=gabb0es&api_key=97d5a64d5ba4a8bc580b752ceff3b87f&format=json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002DC101F8A90>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))
Collected 173 users and 12094 unique songs so far...
Collected 223 users and 14560 unique songs so far...
Collected 272 users and 16780 unique songs so far...
Collected 283 users and 17142 unique songs so far...
Collected 330 users and 19122 unique songs so far...
Collec